# Evaluation of the GAN Greyscale 64 px model

# 1. Import Libaries

In [10]:
import os
import shutil
import sys
import numpy as np
if os.getcwd().split('/')[-1] == 'notebooks':
    os.chdir('../')

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torchvision.datasets as dset
from torchvision import transforms
from torch import autograd
import torch.optim as optim
import torchvision.utils as vutils # draw bounding box, segmantation mask, keypoints. convert to rgb, make grid, save_image

import matplotlib.pyplot as plt

# import ignite
# import ignite.distributed as idist
# from ignite.metrics import FID, InceptionScore, RunningAverage
# from ignite.contrib.handlers import ProgressBar
# from ignite.engine import Engine, Events
# from ignite.handlers import *
import PIL.Image as Image

from src.utils import geometric_score as gs

%matplotlib inline

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
DATAROOT = "./data/cadastralExportRGB/train/"  # root directory for dataset
WORKERS = 4  # number of workers for dataloader
BATCH_SIZE = 2000  # batch size during training
IMG_SIZE = 300  # spatial size of training images (to be resized to)
MULT = 3.15  # re-size factor: 11 if resolution is 64 x 64, 3.15 if resolution is 300 x 300
NC = 3  # number of entities aka channels in the training images
NZ = 512  # size of noise vector (i.e. size of generator input)
NGF = 32  # base size of feature maps in generator
NDF = 32  # base size of feature maps in discriminator
NUM_EPOCHS = 1000  # number of training epochs
LR = 1e-2  # learning rate for both optimizers
DEVICE = 'cpu'
# DEVICE = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")  # which device to run on

In [27]:
x_true = torch.load('./data/test.pkl')
dataloader = torch.utils.data.DataLoader(x_true, batch_size=BATCH_SIZE, 
                                             shuffle=True, num_workers=4, drop_last=True)
x_true = next(iter(dataloader))
x_true = x_true[0].to(DEVICE)

In [19]:
netG = torch.load('./models/netG_dcgan_embeddings.pkl').to(DEVICE)
fixed_noise = torch.randn(2000, NZ, 1, 1, device=DEVICE)
x_pred = netG(fixed_noise)

# Calculating Geometric Score

Implementation: https://github.com/KhrulkovV/geometry-score

In [40]:
rltx = gs.rlts(x_true[:, 1, :, :].view(BATCH_SIZE, -1).numpy(), n=100, L_0=32, i_max=10, gamma=1.0/8)

Done 0/100
Done 10/100
Done 20/100
Done 30/100
Done 40/100
Done 50/100
Done 60/100
Done 70/100
Done 80/100
Done 90/100


In [ ]:
rlty = gs.rlts(x_pred[:, 1, :, :].view(BATCH_SIZE, -1).cpu().detach().numpy(), n=100, L_0=32, i_max=10, gamma=1.0/8)

Done 0/100


In [ ]:
gs.geom_score(rltx, rlty)

In [39]:
gs.geom_score(rltx, rlty)

0.0010152160286518088